# 이변량 범주형 자료 요약

#### 이변량 범주형 자료에 대해 자료의 분할표(contingency table), 카이제곱 검정(chisquare test)을 사용하여 원자료(raw data)의 정리와 요약

- 자료의 형태(질적/범주형변수 vs. 양적변수)에 따라서, 또 양적변수는 연속형이냐 이산형이냐에 따라서 자료를 정리하고 요약하는 방법, 확률모형 설정, 추정과 검정, 모형 개발 등이 달라집니다.

<img src='images/Picture24.png' />

#### 분석 기법은 변수가 몇개냐, 변수의 형태가 무엇이냐(범주형 변수와 연속형 변수의 2개로 구분해서 간략하게 표로 제시해보면 아래와 같습니다.

<img src='images/Picture27.png' />
<img src='images/Picture28.png' />

일변량 자료 분석이 변수 자체의 분포, 형태에 대해 관심을 가진다면, **이변량 자료 분석의 경우는 변수와 변수간의 관계**에 특히 관심을 가지게 됩니다.

#### 사용할 데이터는 Cars93 데이터프레임의 차종형태(Type), 실린더(Cylinders), 트렁크공간크기(Luggage.room)의 변수들입니다. Type, Cylinders 변수는 categorical data 이어서 교차표(crosstabulation table) 분석할 때 그대로 사용하면 되며, Luggage.room 변수는 discrete data (integer) 이므로 categorical data로 변환해서 분할표 분석할 때 사용하겠습니다.

In [104]:
cars = pd.read_csv('data/Cars93.csv').loc[:,('Type','Cylinders','Luggage.room')]

In [98]:
#cars['Luggage'] = np.int64(cars['Luggage.room'].fillna(0))

In [105]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 3 columns):
Type            93 non-null object
Cylinders       93 non-null object
Luggage.room    82 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.3+ KB


#### (1) 이변량 분할표(contingency table)

In [118]:
cars.iloc[:,0:2].head()

,Type,Cylinders
0,Small,4
1,Midsize,6
2,Compact,6
3,Midsize,6
4,Midsize,4


In [169]:
pivot_cars = cars.pivot_table(index='Type', columns='Cylinders', aggfunc=len, fill_value=0)
#cars.pivot_table(index='Type', columns='Cylinders', aggfunc='size', fill_value=0)
pivot_cars

Luggage.room                     
Cylinders            3   4  5   6  8 rotary
Type                                       
Compact              0  15  0   1  0      0
Large                0   0  0   7  4      0
Midsize              0   7  1  12  2      0
Small                3  18  0   0  0      0
Sporty               0   8  0   4  1      1
Van                  0   1  1   7  0      0

#### (2) 상대 분할표 (relative frequency contingency table)

> 구간별 도수를 전체 도수의 총합으로 나눈값

In [178]:
pivot_cars/len(cars)

Luggage.room                                                  
Cylinders            3         4         5         6         8    rotary
Type                                                                    
Compact       0.000000  0.161290  0.000000  0.010753  0.000000  0.000000
Large         0.000000  0.000000  0.000000  0.075269  0.043011  0.000000
Midsize       0.000000  0.075269  0.010753  0.129032  0.021505  0.000000
Small         0.032258  0.193548  0.000000  0.000000  0.000000  0.000000
Sporty        0.000000  0.086022  0.000000  0.043011  0.010753  0.010753
Van           0.000000  0.010753  0.010753  0.075269  0.000000  0.000000

#### (3) 카이제곱 검정(chi-square test)

>**p-value >= 0.05** : 귀무가설(부정적 진술: ~의 차이가 없다) 채택

>**p-value <  0.05** : 귀무가설 기각 -> 대립가설(긍정적 진술: ~의 차이가 있다) 채택

In [179]:
import pandas as pd
from scipy import stats
from numpy import mean

In [457]:
cars = pd.read_csv('data/Cars93.csv').loc[:,('Type','Price','Cylinders')]

In [458]:
cars_pivot = pd.pivot_table(cars, index='Type', columns='Cylinders', aggfunc=len, fill_value=0)
cars_pivot

Price                     
Cylinders     3   4  5   6  8 rotary
Type                                
Compact       0  15  0   1  0      0
Large         0   0  0   7  4      0
Midsize       0   7  1  12  2      0
Small         3  18  0   0  0      0
Sporty        0   8  0   4  1      1
Van           0   1  1   7  0      0

#### 카이제곱 값
>SUM{ [ ( 실제빈도-기대빈도)^2 ] / 기대빈도 } 

#### Yates 검정통계량 (stats.chi2_contingency)
>SUM{ [ ( |실제빈도-기대빈도| - 0.5 )^2 ] / 기대빈도 } 

>여기서, | |는 절대값(absolute value)이라는 의미인데 0.5를 빼기 위해 절대값을 취해야 합니다. 절대값을 취하지 않으면 같은 의미를 가지는 (실제빈도-기대빈도)와 (기대빈도-실제빈도)에 값의 차이가 발생하기 때문입니다. 그리고, 0.5를 빼는 것은 이항분포를 Normal분포에 근사해서 확률을 계산할 때 continuity correction을 하면 이항분포로 계산한 정확한 확률에 더욱 근사하게 되는 것과 유사합니다.

In [460]:
stats.chi2_contingency(cars_pivot)
# p-value 값이 매우 작으므로 차종(Type)과 Cylinder 개수는 
# 5%의 유의수준(significance level)에서 서로 독립이 아니라
# (귀무가설을 기각하고 대립가설을 채택. 서로 연관이 있다고 판단할 수 있다.

(78.93491032776747,
 1.674244316924938e-07,
 25,
 array([[ 0.51612903,  8.43010753,  0.34408602,  5.33333333,  1.20430108,
          0.17204301],
        [ 0.35483871,  5.79569892,  0.23655914,  3.66666667,  0.82795699,
          0.11827957],
        [ 0.70967742, 11.59139785,  0.47311828,  7.33333333,  1.65591398,
          0.23655914],
        [ 0.67741935, 11.06451613,  0.4516129 ,  7.        ,  1.58064516,
          0.22580645],
        [ 0.4516129 ,  7.37634409,  0.30107527,  4.66666667,  1.05376344,
          0.15053763],
        [ 0.29032258,  4.74193548,  0.19354839,  3.        ,  0.67741935,
          0.09677419]]))

#### 문제 A) 4면체 주사위를 3번 던졌다. 만약 3번 모두 1이 나왔다면 주사위가 공정하다는 귀무 가설의 유의 확률은 얼마인가?

In [465]:
x = np.array([3,0,0,0]) # 1이 3번, 2,3,4가 0번 나왔다.
x_exp = np.array([0.75,0.75,0.75,0.75]) # 기대값은 3번/총주사위수4

stats.chisquare(x,x_exp)

Power_divergenceResult(statistic=9.0, pvalue=0.02929088653488826)

##### 위의 p-value 값이 0.02 로 유의수준 0.05 보다 작으므로 귀무가설을 기각하고 대립가설을 채택한다. 즉 유의수준보다 낮다는 의미는 이렇게 낮은 확률로 발생했다는 의미이므로 뭔가 주사위가 공정하지 않다는 것을 뜻한다.

#### 문제 B) 주사위를 60회 던져 1부터 6의 눈을 각각 12, 15, 8, 9, 10, 6 회 얻었다고 해보자. 이 주사위가 특정한 눈이 더 잘 나온다고 할 수 있는가?
* 만약 주사위의 모든 눈이 같은 확률로 나온다고 가정하면 우리는 이론적으로 1부터 6의 눈이 각각 10회씩 나올 것으로 예상할 수 있다. 이것이 기대값 (Expected Value)이다.

In [467]:
x = np.array([12,15,8,9,10,6])
x_exp = np.array([10,10,10,10,10,10])
stats.chisquare(x,x_exp)

Power_divergenceResult(statistic=5.0, pvalue=0.4158801869955079)

##### 위의 결과는 카이제곱값이 5.00, 그리고 p-value 값이  0.4159 이다. 유의수준을 0.05로 보았을 때 0.4159가 0.05 보다 크기에 귀무가설을 기각할 수 없다. 즉 위의 주사위는 특정한 눈이 더 잘 나온다고 할 수 없다.

---

#### 참고
확률 분포 객체를 생성하는 명령(Scipy.stats)
종류    이름         확률 분포

이산 - bernoulli - 베르누이 분포

이산 - binom - 이항 분포

연속 - uniform - 균일 분포

연속 - norm - 가우시안 정규 분포

연속 - beta - 베타 분포

연속 - gamma - 감마 분포

연속 - t - 스튜던트 t 분포

연속 - chi2 - 카이 제곱 분포

연속 - f - F 분포